# Import Libraries & Custom Function

In [2]:
from itertools import combinations 
import pickle
import pandas as pd
import numpy as np
!pip install pyfpgrowth
import pyfpgrowth as pyfp
import pandas as pd
from sklearn.externals import joblib
from sagemaker import get_execution_role
import boto3
import math
from s3fs.core import S3FileSystem
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_column', None)
pd.set_option('display.max_row', None)

def rSubset(arr, r): 
  
    # return list of all subsets of length r 
    # to deal with duplicate subsets use  
    # set(list(combinations(arr, r))) 
    return list(combinations(arr, r)) 

def insertUniquePairingIntoDict(dic, array):
    for pair in rSubset(array, 2):
        if dic.get(pair) == None:
            dic[pair] = 1
        else:
            dic[pair] = dic[pair] + 1
    return dic
    
def openPickle(name):
    file = open(name, 'rb')
    return pickle.load(file)

    100% |████████████████████████████████| 1.6MB 19.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for pyfpgrowth ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/3b/3f/0d/a04bb8b17887c1eca7d0f1a48d4aa0c09c96eb221ff7fa56c1
Successfully built pyfpgrowth
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Turning Our Dictionary into a Matrix 

In [3]:
test = openPickle("productMatrix.pickle")


In [4]:
freqMatrix = np.empty([49689, 49689], dtype=int)
freqMatrix[0][0]

0

## Adding the values into the matrix

In [5]:
for keys in test.keys():
    freqMatrix[keys[0]][keys[1]] += test[keys]
    freqMatrix[keys[1]][keys[0]] += test[keys]


## Get Similarity Angle Between Two Products

In [7]:
def getSimilarity(product1,product2):
    y1 = 0
    y2 = 0
    for freq in freqMatrix[product1]:
        y1 += (freq * freq)
    for freq in freqMatrix[product2]:
        y2 += (freq * freq)
    x = 0
    for i in range(0,49689):
        if (freqMatrix[product1][i] > 0) & (freqMatrix[product2][i] > 0):
            x += freqMatrix[product1][i] * freqMatrix[product2][i]
    return x / (math.sqrt(y1) * math.sqrt(y2))
    
# getSimilarity(21137,24852)


## Get Similarities Between ALL Products

In [20]:
def getAllSimilarities(i1):
    similaritiesMatrix = {}
    for i2 in freqMatrix[i1]:
        if (freqMatrix[i1][i2] > 0) & (similaritiesMatrix.get((i1,i2)) == None):
            similaritiesMatrix[(i1,i2)] = getSimilarity(i1,i2)
    return similaritiesMatrix

## Get Top 10 Closest Similarity

In [44]:

def getTop10(simDictionary):
    # Create a list of tuples sorted by index 1 i.e. value field     
    sortedSimDictionary = sorted(simDictionary.items(), reverse=True ,  key=lambda x: x[1])
 
    # Iterate over the sorted sequence
    top10Items = {}
    iCount = 0
    for elem in sortedSimDictionary :
        if iCount == 10:
            break
        #print(elem[0] , " ::" , elem[1] )
        top10Items[elem[0][1]] = elem[1]
        iCount += 1
    return top10Items

## Getting Top 10 For Demo Catalog Items

In [85]:
catalog = {}
catalogIds = [24852,204,21903,44142,44500,44632,20479,34050,9804,20126,2144]
for ids in catalogIds:
    catalog[ids] = getTop10(getAllSimilarities(ids))

## Save the Catalog

In [87]:
with open('catalogPart1.pickle', 'wb') as handle:
    pickle.dump(catalog, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Add Missing Item from catalog

In [ ]:
catalogTemp = {}
catalogTemp[24699] = getTop10(getAllSimilarities(24699))
catalogTemp[24699]
#with open('catalogPart1.pickle', 'wb') as handle:
    #pickle.dump(catalog, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [93]:
with open('catalogPart3.pickle', 'wb') as handle:
    pickle.dump(catalogTemp, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [51]:
#freqMatrixDictionary = {}
#for i1 in range(0,49689):
   # for i2 in freqMatrix[i1]:
       # if (freqMatrix[i1][i2] > 0) & freqMatrixDictionary[(i1,i2)] == None:
          #  freqMatrixDictionary[(i1,i2)] = getSimilarity(i1,i2)

In [ ]:
#with open('freqMatrixDictionary.pickle', 'wb') as handle:
   # pickle.dump(freqMatrixDictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)